# Vault Setup

This tutorial has three ways of starting up Vault.

1. Vault OSS via a Docker container
1. Vault Enterprise via a Docker container
1. Vault via binary on localhost

## Pre-requisites

- docker
- docker compose
- jq
- vault

### Set Up Environment Variables

Some env

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"

# Some commands may have sensitive information. Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history

# Set dir env vars. 
MAIN_DIR=$(pwd)
WORK_DIR=/tmp/config/vault

Customize the values below if you want Vault to start with different parameters

In [ ]:
# Common
export VAULT_VER=1.7.5 # 1.8+ enterprise requires license file; 1.7.2 has 8 hour eval
export VAULT_PORT=8200
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
export VAULT_LICENSE=$(cat ../../license/vault.hclic)

In [ ]:
# Create required directories.
mkdir -p $WORK_DIR/{data,logs,config}

## Container
This sets up Vault in a Docker container. Run Vault with either docker compose or the binary version below.

> Don't try both at the same time, unless you modify the ports used first.

> I prefer the docker method. It allows me to easily test different versions of Vault. Adding additional components such as consul, web, db, etc is easier as well with containers.

### Vault Container Configuration

There are two option to choose below.

* OSS vs Enterprise
  * Vault OSS
	* comment out this line `image: hashicorp/vault-enterprise`
	* uncomment this line `image: hashicorp/vault`
  * Vault Enterprise
	* uncomment this line `image: hashicorp/vault-enterprise`
	* comment out this line `image: hashicorp/vault`
* Dev Mode vs local config (requires initialization)
  * Dev mode
	* uncomment this line `entrypoint: "vault server -dev`
    * comment out this line `entrypoint: "vault server -config=/vault/config"` and `- ./config/vault/config:/vault/config:ro`
  * Local config
	* comment out this line `entrypoint: "vault server -dev`
    * uncomment this line `entrypoint: "vault server -config=/vault/config"` and `- ./config/vault/config:/vault/config:ro`


The default is to run Vault Enterprise in Dev Mode.

Create a `docker-compose.yaml` file.

In [ ]:
## Create docker-compose file.
cat > docker-compose.yaml << EOF
version: '3.8'
services:
  vault:
    image: hashicorp/vault-enterprise:${VAULT_VER}_ent # Vault Enterprise
    #image: hashicorp/vault:${VAULT_VER}                # Vault OSS
    container_name: vault
    restart: always
    volumes:
#      - ./${WORK_DIR}/data:/vault/data # uncomment to persist data
      - ./${WORK_DIR}/logs:/vault/logs
      # - ./${WORK_DIR}/config:/vault/config:ro # uncomment for local config
    ports:
      - "$VAULT_PORT:8200/tcp"
    environment:
      VAULT_DEV_ROOT_TOKEN_ID: ${VAULT_TOKEN}
      VAULT_DEV_LISTEN_ADDRESS: "0.0.0.0:8200"
      VAULT_ADDR: ${VAULT_ADDR}
      VAULT_LICENSE: ${VAULT_LICENSE}
    cap_add:
      - IPC_LOCK
    entrypoint: "vault server -dev" # dev mode
    #entrypoint: "vault server -config=/vault/config" # non-dev with local config
EOF

### Vault Configuration - for non-Dev Mode (WIP)

If want to run with a local config, then create a Vault config file. This file is placed in a folder mounted by the container.

In [ ]:
cat > $WORK_DIR/config/vault1.hcl << EOF
# storage "file" {
#     path = "/vault/file"
# }

storage "raft" {
  path    = "./vault/data"
  node_id = "node1"
}

listener "tcp" {
  address       = "0.0.0.0:8200"
#   tls_cert_file = "/path/to/fullchain.pem"
#   tls_key_file  = "/path/to/privkey.pem"
  tls_disable = true
  telemetry {
    unauthenticated_metrics_access = true
  }
}

default_lease_ttl = "168h" # default(768h)
max_lease_ttl = "0h" # default(768h)
api_addr = "http://0.0.0.0:8200"
cluster_addr = "https://0.0.0.1:8201"
ui = true

log_level = debug
telemetry {
  prometheus_retention_time = "24h"
  disable_hostname = true
}
EOF

### Start Container(s)

In [ ]:
docker-compose up --force-recreate -d 2>&1 \
  | tee /tmp/vault_docker_compose.log

In [ ]:
vault status
curl http://localhost:8200/v1/sys/seal-status | jq

## localhost - binary

Install Vault on Mac or Linux.

In [ ]:
HASHI_RELEASES=https://releases.hashicorp.com
VAULT_VER=1.10.4   # 1.8+ enterprise requires license; no timed eval
VAULT_ENT=""       #"+ent" or ""
VAULT_OS="darwin"  # "darwin" or "linux"

In [ ]:
curl -O ${HASHI_RELEASES}/vault/${VAULT_VER}${VAULT_ENT}/vault_${VAULT_VER}${VAULT_ENT}_darwin_amd64.zip \
  && unzip vault_${VAULT_VER}${VAULT_ENT}_darwin_amd64.zip \
  && mv vault /usr/local/bin \
  && rm vault_${VAULT_VER}${VAULT_ENT}_darwin_amd64.zip

In [ ]:
vault version

This sets up Vault on the local host. Run either this or the Docker version above. 

In [ ]:
vault server \
  -dev -dev-root-token-id=${VAULT_TOKEN} \
  -dev-listen-address=0.0.0.0:8200 \
  -log-level=trace \
  -config=./config.hcl > ${WORK_DIR}/vault.log 2>&1 &

In [ ]:
cat ${WORK_DIR}/vault.log

## Vault - Initialize and Unseal

For non-Dev mode, you will need to initialize and unseal your cluster.

In [ ]:
vault operator init -key-shares=1 -key-threshold=1 > $WORK_DIR/vault.init

In [ ]:
VAULT_TOKEN=$(grep "Root Token" $WORK_DIR/vault.init | awk '{print $NF}')
UNSEAL_KEY1=$(grep "Key 1" $WORK_DIR/vault.init | awk '{print $NF}')

In [ ]:
vault operator unseal ${UNSEAL_KEY1}

## Status, Audit, and License

In [ ]:
vault status

Enable audit output to files with and without hashing. Do not leave `log_raw` on in production.

In [ ]:
# Enable audit output to file, audit settings are in vault1.hcl
vault audit enable file file_path=/vault/logs/audit.log
vault audit enable file file_path=/tmp/audit.log #works in container or localhost
vault audit enable -path=raw file file_path=/vault_logs/audit_raw.log log_raw=true

Install license via API. Required for Vault Enterprise prior to 1.8.

In [ ]:
curl \
    --silent \
    --header "X-Vault-Token: $VAULT_TOKEN" \
    --request PUT \
    --data '{"text":"'$VAULT_LICENSE'"}' \
    $VAULT_ADDR/v1/sys/license

Check status again. Make sure vault is Initialized and Unsealed.

In [ ]:
vault status

## UI

In [ ]:
if [[ $(uname -s) == "Linux" ]]; then
  printf "Go to http://127.0.0.1:8200\n"
elif [[ $(uname) == "Darwin" ]]; then
  open $VAULT_ADDR
else
  printf "Go to http://127.0.0.1:8200\n"
fi # mac only
printf "${GREEN}Login with token:${NC} $VAULT_TOKEN"

## Sample Authentication Methods

* For LDAP configuration, see [LDAP](./110-Setup-authmethods.ipynb#LDAP)
* For User Pass, see [UserPass](./110-Setup-authmethods.ipynb#UserPass)

## Clean Up

### Clean up Docker

In [ ]:
docker compose down || true #mac
docker-compose down #linux
docker ps | grep -i vault

### Clean up localhost

In [ ]:
pkill vault

### Clean up artifacts

In [ ]:
rm -rf /tmp/config/vault/*
rm /tmp/audit.log /tmp/vault_docker_compose.log
rm -rf docker-compose.log docker-compose.yaml